In [1]:
#This is for aggregated insurence state wise data
import pandas as pd
import json
import os

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/aggregated/insurance/country/india/state"
Agg_state_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'State':[], 'Year':[],'Quater':[],'Insurance':[], 'Insurance_count':[], 'Insurance_amount':[]}

for i in Agg_state_list:
    p_i=path+"/"+i
    Agg_yr=os.listdir(p_i)
    #delhi
    for j in Agg_yr:
        p_j=p_i+"/"+j
        #2018
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+"/"+k 
            #1.json
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['transactionData']:
              Name=z['name']
              count=z['paymentInstruments'][0]['count']
              amount=z['paymentInstruments'][0]['amount']
              clm['Insurance'].append(Name)
              clm['Insurance_count'].append(count)
              clm['Insurance_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
#Creating dataframe
Agg_Insu_state=pd.DataFrame(clm)

print(Agg_Insu_state)

                         State  Year  Quater  Insurance  Insurance_count  \
0    andaman-&-nicobar-islands  2020       2  Insurance                6   
1    andaman-&-nicobar-islands  2020       3  Insurance               41   
2    andaman-&-nicobar-islands  2020       4  Insurance              124   
3    andaman-&-nicobar-islands  2021       1  Insurance              225   
4    andaman-&-nicobar-islands  2021       2  Insurance              137   
..                         ...   ...     ...        ...              ...   
677                west-bengal  2023       4  Insurance            72712   
678                west-bengal  2024       1  Insurance            79576   
679                west-bengal  2024       2  Insurance            67048   
680                west-bengal  2024       3  Insurance            77158   
681                west-bengal  2024       4  Insurance            91719   

     Insurance_amount  
0              1360.0  
1             15380.0  
2            15

In [ ]:
#This is for aggregated insurance for the year 2020 data
import pandas as pd
import json
import os

#This is to direct the path to get the data from 2020

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/aggregated/insurance/country/india/2020"
Agg_insu_2020_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'Quater':[],'Transacion_type':[], 'Transacion_count':[], 'Transacion_amount':[]}

for i in Agg_insu_2020_list:
    p_i=path+"/"+i
    Data=open(p_i,'r')
    D=json.load(Data)
    for z in D['data']['transactionData']:
        Name=z['name']
        count=z['paymentInstruments'][0]['count']
        amount=z['paymentInstruments'][0]['amount']
        clm['Transacion_type'].append(Name)
        clm['Transacion_count'].append(count)
        clm['Transacion_amount'].append(amount)
        clm['Quater'].append(int(i.strip('.json')))

#Creating dataframe
Agg_insu_2020=pd.DataFrame(clm)
print(Agg_insu_2020)

   Quater Transacion_type  Transacion_count  Transacion_amount
0       2       Insurance            185348         33732166.0
1       3       Insurance            354284         89495076.0
2       4       Insurance            248626        170979933.0


In [ ]:
#creating table to store the above data
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="xxx", #give your user name. Mostly its root
    password="xxx", #give your password
    database="xxx" #give any database name you want
)
cursor=connection.cursor()
query1 = """
        create table Aggregated_Insurance_State(
        State varchar(100),
        Year int,
        Quater int,
        Insurance varchar(100),
        Insurance_count int,
        Insurance_amount float)
        """
cursor.execute(query1)

In [4]:
#storing the above in mysql database
#all data must be tuple and must be a list of tuples
data=[]
for index in Agg_Insu_state.index:
    row = Agg_Insu_state.loc[index]
    row = row.values
    row = row[0],int(row[1]),int(row[2]),row[3],int(row[4]),float(row[5])
    data.append(row)
query2 = "insert into Aggregated_Insurance_State values (%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [2]:
#doing data cleaning for Agg_Insu_state
from sklearn.preprocessing import LabelEncoder
#first step check and remove null values
Agg_Insu_state.isna().sum()
Agg_Insu_state.info()
#Second do encoding and dummies to make all values int
enc_col = ['Insurance']
dummy_col = ['Year','State','Quater']
encoder = LabelEncoder()
Agg_Insu_state['encoded_insurance'] = encoder.fit_transform(Agg_Insu_state['Insurance'])
Agg_Insu_state = pd.get_dummies(Agg_Insu_state,columns=dummy_col,dtype=int)
#third removing outliners
Agg_Insu_state['insurance_count_std']=(Agg_Insu_state['Insurance_count']-Agg_Insu_state['Insurance_count'].mean())/Agg_Insu_state['Insurance_count'].std()
Agg_Insu_state['insurance_amount_std']=(Agg_Insu_state['Insurance_amount']-Agg_Insu_state['Insurance_amount'].mean())/Agg_Insu_state['Insurance_amount'].std()
condition1 = Agg_Insu_state['insurance_count_std']<=3
condition2 = Agg_Insu_state['insurance_count_std']>=-3
condition3 = Agg_Insu_state['insurance_amount_std']<=3
condition4 = Agg_Insu_state['insurance_amount_std']>=-3
Agg_Insu_state = Agg_Insu_state[(condition1) & (condition2) & (condition3) & (condition4)]
print(Agg_Insu_state)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682 entries, 0 to 681
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             682 non-null    object 
 1   Year              682 non-null    object 
 2   Quater            682 non-null    int64  
 3   Insurance         682 non-null    object 
 4   Insurance_count   682 non-null    int64  
 5   Insurance_amount  682 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 32.1+ KB
     Insurance  Insurance_count  Insurance_amount  encoded_insurance  \
0    Insurance                6            1360.0                  0   
1    Insurance               41           15380.0                  0   
2    Insurance              124          157975.0                  0   
3    Insurance              225          244266.0                  0   
4    Insurance              137          181504.0                  0   
..         ...           

In [7]:
!pip3 install plotly


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip3 install --upgrade nbformat


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#You need to map your State names to the names used in the GeoJSON file (properties.ST_NM). Otherwise you will get blank figure.
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
Agg_Insu_state['State'] = Agg_Insu_state['State'].str.strip().str.lower()
Agg_Insu_state['State'] = Agg_Insu_state['State'].replace(state_name_map)

In [ ]:
#plotting aggregated_state_insurance in the form of india map using plotly
import plotly.express as px

fig = px.choropleth(
    Agg_Insu_state,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='Insurance_count',
    color_continuous_scale='Reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()
#if you don't do dummy encoding for state, you will get output